# Оценка линейного искажающего оператора в задаче восстановления изображений

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
# import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

# from skimage import color, data, restoration
# from skimage.restoration import uft
# from scipy.signal import fftconvolve, convolve, convolve2d
# from scipy.stats.stats import pearsonr
# from numpy.fft import fftn, ifftn
# from skimage.measure import compare_psnr
# from skimage.draw import bezier_curve
# import scipy.ndimage.filters as fi
# from itertools import chain
# from skimage.draw import line_aa
# from math import sin, cos, pi
from functools import partial
from scipy.optimize import minimize

from util import *
from richardson_lucy import *

In [ ]:
#interactive graphics
#%matplotlib notebook

In [ ]:
# Prepare data
sz = 11
astro = color.rgb2gray(data.astronaut())

psf = np.ones((sz, sz)) / (sz*sz)
astro_noisy = conv2(astro, psf, 'same')
# Add Noise to Image
#astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

In [ ]:
g = gkern2(100,20)
plt.imshow(g, 'gray')
plt.show()

In [ ]:
taperPSF = gkern2(60, 30)
astro_noisy2 = edgetaper(astro, taperPSF)
#plt.figure(figsize=(10,10))
plt.imshow(astro_noisy2, 'gray')
plt.show()

In [ ]:
# Restore Image using Richardson-Lucy algorithm:
iterations = 7
taperPSF = gkern2(60, 30)
astro_noisy2 = edgetaper(astro_noisy, taperPSF)
deconv = richardson_lucy_matlab(astro_noisy2, psf, iterations=iterations, eps=1e-5)
show_results(astro, astro_noisy, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
liftingbody = img_as_float(io.imread('liftingbody.png'))
lifting_blurred = conv2(liftingbody, psf, 'same')
taper_psf = gkern2(60,30)
lifting_blurred = edgetaper(lifting_blurred, taper_psf)

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 40
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=False)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

// matlab original for gen psf (curved line shift)
```python
def gen_psf():
    p0 = np.random.randint(6, 25)
    p1 = np.random.randint(6, 21)

    x = np.random.randint(0, p0, (1,4))#sort(randi([0 p0], 1, 4));
    y = np.random.randint(0, p1, (1,4))

    pt1 = [x[0]; y[0]]
    pt2 = [x[1]; y[1]]
    pt3 = [x[2]; y[2]]
    pt4 = [x[3]; y[3]]

    t = list(range(0, p0+1))
    pts = kron((1-t).^3,pt1) + kron(3*(1-t).^2.*t,pt2) + kron(3*(1-t).*t.^2,pt3) + kron(t.^3,pt4);
    x1 = 1:p0;
    y1 = round(pts(2, :));
    y1 = y1 - min(y1) + 1;
    %plot(x1,y1);
    pp = max(y1);
    PSF = zeros(p0, pp);

    for ind_x = 1:length(x1)
        yy = max(length(x1) - y1(ind_x), 1);
        PSF(yy, ind_x) = rand();
    end

    PSF = PSF./(sum(PSF(:)) + eps*p0*p0);
```

## Криволинейный оператор смаза

In [ ]:
psf = curved_psf()
lifting_blurred = convolve2d(liftingbody, psf, 'same')

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 3
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=False, dampar=0.004225632509230561)
deconv_py = restoration.richardson_lucy(lifting_blurred, psf, iterations=iterations)
show_results(lifting_blurred, deconv_py, deconv['image'], titles=['Blurred data', 'Restoration using\nRichardson-Lucy python', 'Restoration using\nRichardson-Lucy my'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

## Линейный оператор смаза

In [ ]:
np.round(motion_blur_psf_my(x=3,y=-4.5),3)

In [ ]:
shift = 30
psf = motion_blur_psf_my(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 12
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=0.004)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])
correlation_X = pearsonr(liftingbody.ravel('C')[:-1], liftingbody.ravel('C')[1:])
correlation_Y = pearsonr(liftingbody.ravel('F')[:-1], liftingbody.ravel('F')[1:])
print(correlation_X, correlation_Y)

## Неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/4)
#iterations = 20
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

## Использование параметра dampar

In [ ]:
s_n, S_find = find_noise(liftingbody)
s_n

In [ ]:
psf = motion_blur_psf(shift, pi/4)
x0_ref = [shift * np.cos(pi/4), shift * np.sin(pi/4)]
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=False, dampar=s_n, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

### dampar и неверная psf

In [ ]:
psf_wrong = motion_blur_psf_my(shift, pi/3)
#iterations = 40
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

```
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = lifting_blurred.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
```

## Кепстр
$$K = F^{-1}\{log(1+\left|F\{I\}\right|)\}$$

In [ ]:
b_clip = 3
N,M = lifting_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_blurred))))#[b_clip:N//2,b_clip:M//2]
K_shift = np.fft.fftshift(K)
#K = gaussian(np.abs(K), 1)

#mask = np.ones((N, M))
#mask[1:3, 1:3] = 0
#K *= mask

In [ ]:
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


h, w = K.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)
print(X.shape, Y.shape)
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
H = 1
surf = ax.plot_surface(X, Y, np.abs(K_shift), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel('x')
plt.ylabel('y')
# Customize the z axis.
#ax.set_zlim(0, H + H/20)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = [nn // n - n//2, nn % n - m//2]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

In [ ]:
x0

## Уточнение искажающего оператора
$$\varepsilon = ||\tilde{I}\oplus\tilde{h}-I_0|| \to \min_{(x,y)}$$
Пусть $$\tilde{I}\oplus(\tilde{h}+\tilde{dh})=I_0$$
Тогда $$\tilde{I}\oplus\tilde{dh}=I_0-\tilde{I}\oplus\tilde{h}$$
Получим задачу аналогичную исходной($\tilde{I}\oplus h +\eta = I_0$)

```python
img_diff = liftingbody - convolve2d(deconv['image'], psf, mode='same') # Утечка
deconv_psf = richardson_lucy_matlab(img_diff, deconv['image'], iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
psf_new = deconv_psf['image']
deconv_upd = richardson_lucy_matlab(lifting_blurred, psf_new, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
show_results(lifting_blurred, deconv['image'], deconv_upd['image'],
             titles=['blurred', 'restored', 'restored with\nnew psf'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_upd['correlationX'],
                         deconv_upd['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(new psf)', 'column correlation(new psf)'])
```

Такой подход не сработал :( Будем использовать метод Ньютона
## Уточнение искажающего оператора

$$(x,y)^T_{n+1} = (x, y)^T_{n} - H^{-1}(f(x,y))\nabla f(x,y)$$
$$H(f) =$$

In [ ]:
def funcToMinimize(xy, I_blurred, *args, **kwargs):
    psf = motion_blur_psf_my(x=xy[0], y=xy[1])
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df))

In [ ]:
partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([10,10])

In [ ]:
cos(pi/4)*30

### Минимизация методом Нелдера-Мида (симплекс-метод)

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='nelder-mead',
               options={'xtol': 1e-3, 'disp': True})
```

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='powell',
               options={'xtol': 1e-3, 'disp': True})
```

## Метод Ньютона

In [ ]:
res_tnc = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='TNC')
print(res_tnc)

In [ ]:
res_tnc['x']

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = motion_blur_psf_my(x=x0[0], y=x0[1])
deconv_init = richardson_lucy_matlab(lifting_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

x_found, y_found = res_tnc['x']
psf_found = motion_blur_psf_my(x=x_found, y=y_found)
deconv_found = richardson_lucy_matlab(lifting_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, x0_ref), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_tnc['x'])])

## Оценка криволинейного оператора смаза, представленного кривой Безье

In [ ]:
sz = 20
#points_real = [2*sz//3-1, sz//3, sz-1, sz-1]
points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier)
plt.show()

In [ ]:
psf_bezier

```
import math
def bezier2_psf(points):
    print(points)
    #points = list(map(int, points))
    shape = math.ceil(max(points[1::2]))+1, math.ceil(max(points[::2]))+1
    r, c = bezier_curve(0, 0, *points, weight=1)
    psf_bezier = np.zeros(shape)
    psf_bezier[r,c] = 1
    psf_bezier /= psf_bezier.sum()
    return psf_bezier
```

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')

lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
psnr_py = compare_psnr(liftingbody, lifting_restored)

deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=True, dampar=0.004)
psnr_my = compare_psnr(liftingbody, deconv_bezier['image'])

In [ ]:
show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
            titles=['Blurred image', 'Restored with python LR\nPSNR={0}'.format(psnr_py), 'Restored with my LR\nPSNR={0}'.format(psnr_my)])

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, *args, **kwargs):
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df))
partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([0, 10, 10, 10])

In [ ]:
x0 = [20, 0, 15, 20]
res_tnc = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                   x0=x0, method='TNC')
print(res_tnc)

# [(12.0, 10.0), (20.0, 20.0)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.00000001, 10.0), (20.0, 20.0)]
# [(12.0, 10.00000001), (20.0, 20.0)]
# [(12.0, 10.0), (20.00000001, 20.0)]
# [(12.0, 10.0), (20.0, 20.00000001)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.00000001, 10.0), (20.0, 20.0)]
# [(12.0, 10.00000001), (20.0, 20.0)]
# [(12.0, 10.0), (20.00000001, 20.0)]
# [(12.0, 10.0), (20.0, 20.00000001)]
#      fun: 0.00502600166295737
#      jac: array([0., 0., 0., 0.])
#  message: 'Local minimum reached (|pg| ~= 0)'
#     nfev: 1
#      nit: 0
#   status: 0
#  success: True
#        x: array([12., 10., 20., 20.])
# ```

In [ ]:
points_found = res_tnc['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)


In [ ]:
x0 = [12, 10, 20, 20]
res_nelder = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True), x0=x0, method='Nelder-Mead', options={'xtol': 1e-3, 'disp': True})
print(res_nelder)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_nelder['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, x0_ref), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_nelder['x'])])

* Запустить градиентный метод
* Сделать мощный edgetaper
* Сделать антиалиасинг psf